# Γκολφ, Θερισμός, Ομαλοποίηση

Προσαρμοσμένο από [την αντίστοιχη τεκμηρίωση του TensorFlow](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit).

---

> Πάνος Λουρίδας, Αναπληρωτής Καθηγητής <br />
> Τμήμα Διοικητικής Επιστήμης και Τεχνολογίας <br />
> Οικονομικό Πανεπιστήμιο Αθηνών <br />
> louridas@aueb.gr

In [ ]:
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

tf.random.set_seed(0) # for replicability purposes, not for production

* Τα δεδομένα που θα χρησιμοποιήσουμε προέρχονται από μία έρευνα όπου εξετάστηκαν οι δυνατότητες νευρωνικών δικτύων στην αναζήτηση νέων σωματιδίων στη φυσική: Baldi, P., Sadowski, P. & Whiteson, D. Searching for exotic particles in high-energy physics with deep learning. Nat Commun 5, 4308 (2014). https://doi.org/10.1038/ncomms5308.

* Το σύνολο των δεδομένων είναι διαθέσιμο από το <https://archive.ics.uci.edu/ml/datasets/HIGGS>. Εμείς για πρακτικούς λόγους θα χρησιμοποιήσουμε ένα υποσύνολο με 13.000 δείγματα.

* Ο σκοπός είναι να κάνουμε ταξινόμηση μεταξύ δύο κλάσεων (στήλη `label`, αν υπάρχει ένδειξη ή όχι) με βάση 28 χαρακτηριστικά.

In [ ]:
higgs_data = pd.read_csv("data/higgs.csv", dtype=float)
higgs_data

* Θα πάρουμε 11.000 δείγματα για εκπαίδευση και επικύρωση και θα χρησιμοποιήσουμε 2.000 δείγματα για έλεγχο.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(higgs_data.iloc[:, 1:], 
                                                    higgs_data.iloc[:, 0], 
                                                    train_size=11000)

* Θα εξασφαλίσουμε ότι οι τιμές των χαρακτηριστικών έχουν μέση τιμή μηδέν και διακύμανση ένα.

In [ ]:
normalizer = layers.Normalization()
normalizer.adapt(np.array(X_train))

* Στο μοντέλο μας θα χρησιμοποιήσουμε μία συνάρτηση ενεργοποίησης που δεν έχουμε ξαναδεί, την Εκθετική Γραμμική Μονάδα, ELU (Exponential Linear Unit):

$$
  f(x) = 
  \begin{cases} 
   x &  x > 0 \\
   \alpha (e^{x} - 1) &  x \le 0
  \end{cases}
$$

* Η ELU διαφέρει από τη ReLU στο ότι είναι πιο ομαλή κοντά στο σημείο μηδέν και επιπλέον μπορεί να δώσει αρνητική έξοδο.

In [ ]:
plt.rcParams["text.usetex"] = True

elu_label = (
    r"\begin{eqnarray*}"
    r"f(x) &=& x \quad x > 0 \\"
    r"f(x) &=& \alpha (e^{x} - 1)\quad  x \le 0"
    r"\end{eqnarray*}"
)
fig = plt.figure(figsize=(8, 6))
ax = plt.axes()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.xlim((-5, 5))
plt.xticks(np.arange(-5, 6, 1))
x = np.linspace(-5, 5, 100)
y = np.zeros_like(x)
y = x.copy()
alpha = 1
y[0:50] = alpha * (np.exp(x[0:50]) - 1)
ax.text(-4.5, 5, elu_label, color="k", fontsize=16)
_ = plt.plot(x, y)

* Θα ορίσουμε τον αριθμό των εποχών, το μέγεθος κάθε φουρνιάς, και από εκεί τον αριθμό των βημάτων εκπαίδευσης σε κάθε εποχή.

In [ ]:
NUM_EPOCHS = 10000
BATCH_SIZE = 100
STEPS_PER_EPOCH = X_train.shape[0] // BATCH_SIZE

* Μία σημαντική υπερπαράμετρος στα νευρωνικά δίκτυα είναι ο *ρυθμός εκμάθησης* (learning rate).

* Αυτός ορίζει πόσο μεγάλες θα είναι οι διορθώσεις που γίνονται στα βάρη και στις πολώσεις σε κάθε βήμα της εκμάθησης.

* Αν ο ρυθμός εκμάθησης είναι μεγάλος, τότε οι διορθώσεις είναι μεγάλες, και το δίκτυο μαθαίνει γρήγορα.

* Από την άλλη, μπορεί να είναι «απρόσεχτο»: οι διορθώσεις να είναι τόσο μεγάλες, ώστε να ξεφύγει από τις βέλτιστες τιμές των βαρών και των πολώσεων.

* Αν ο ρυθμός εκμάθησης είναι μικρός, τότε οι διορθώσεις είναι μικρότερες, το δίκτυο μαθαίνει πιο συντηρητικά, και χρειάζεται περισσότερος χρόνος για την εκπαίδευση.

* Ένας τρόπος να το προσεγγίσουμε αυτό είναι να σκεφτούμε τι γίνεται στο γκολφ.

* Στην αρχή ο παίκτης είναι μακριά από την τρύπα και τα χτυπήματά του θα είναι δυνατά, ώστε γρήγορα η μπάλα να προσεγγίσει την τρύπα.

* Όταν όμως πλησιάσει στην τρύπα, ο παίκτης γίνεται πιο προσεκτικός. Αν συνεχίσει να χτυπάει με τον ίδιο τρόπο, η μπάλα απλώς θα φεύγει από την άλλη μεριά.

* Αν χρησιμοποιήσουμε τον βελτιστοποιητή Adam (ή και άλλους), μπορούμε να ορίσουμε ένα μεταβλητό ρυθμό εκμάθησης.

* Συγκεκριμένα, θα ορίσουμε ένα *πρόγραμμα εκμάθησης* (learning schedule), στο οποίο o ρυθμός εκμάθησης θα μειώνεται βαθμιαία.

* Η ιδέα είναι ότι αρχικά θέλουμε το νευρωνικό μας δίκτυο να μαθαίνει γρήγορα, αλλά όσο προχωράει η εκπαίδευση θέλουμε να γίνεται πιο προσεκτικό.

* Χρησιμοποιώντας την κλάση `InverseTimeDecay` ορίζουμε ότι θα ξεκινήσουμε με έναν αρχικό ρυθμό εκμάθησης και στη συνέχεια κάθε 100 εποχές αυτός θα μειώνεται ώστε στις 100 εποχές να γίνει 1/2 του αρχικού, στις 200 το 1/3 του αρχικού, κ.ο.κ.

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH * 100,
  decay_rate=1,
  staircase=False)

In [ ]:
step = np.linspace(0,100000)
lr = lr_schedule(step)
plt.figure(figsize = (8,6))
plt.plot(step/STEPS_PER_EPOCH, lr)
plt.ylim([0,max(plt.ylim())])
plt.xlabel('Epoch')
_ = plt.ylabel('Learning Rate')

* Φτιάχνουμε τώρα το μοντέλο μας, όπου θα χρησιμοποιήσουμε όπως στην αρχική δημοσίευση στρώματα των 300 νευρώνων.

* Στο τέλος θα έχουμε ένα στρώμα με σιγμοειδή συνάρτηση ενεργοποίησης, αφού θέλουμε ταξινόμηση μεταξύ δύο κλάσεων.

* Η απώλεια θα υπολογίζεται μέσω της κλάσης `BinaryCrossentropy` και ως μετρικές θα έχουμε την`BinaryAccuracy` και την `BinaryCrossEntropy` (θα επανέλθουμε αργότερα σε αυτό το σημείο).

In [ ]:
model = keras.Sequential([
    normalizer,
    layers.Dense(300, activation='elu'),
    layers.Dense(300, activation='elu'),
    layers.Dense(300, activation='elu'),    
    layers.Dense(300, activation='elu'),
    layers.Dense(300, activation='elu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(lr_schedule),
              metrics=[
                  tf.keras.losses.BinaryCrossentropy(from_logits=False, name='binary_crossentropy'),
                  tf.metrics.BinaryAccuracy(threshold=0.5)
              ])

* Να το δούμε συνοπτικά.

In [ ]:
model.summary()

* Θα εκπαιδεύσουμε το μοντέλο μας μέχρι 10000 εποχές.

* Αντί να εμφανίζουμε την πρόοδο ανά μία εποχή (πάρα πολλές), θα εμφανίζουμε την πρόοδο ανά 100 εποχές, ενώ κάθε μία εποχή που ολοκληρώνεται θα τυπώνουμε απλώς μια τελεία.  

* Για να το κάνουμε αυτό θα ορίσουμε μία δική μας κλάση που θα δώσουμε στο TensorFlow ως callback.

In [ ]:
# From https://github.com/tensorflow/docs/blob/master/tools/tensorflow_docs/modeling/__init__.py
class EpochDots(tf.keras.callbacks.Callback):
    """A simple callback that prints a "." every epoch, with occasional reports.
    
    Args:
        report_every: How many epochs between full reports
        dot_every: How many epochs between dots.
    """

    def __init__(self, report_every=100, dot_every=1):
        self.report_every = report_every
        self.dot_every = dot_every

    def on_epoch_end(self, epoch, logs):
        if epoch % self.report_every == 0:
            print()
            print('Epoch: {:d}, '.format(epoch), end='')
            for name, value in sorted(logs.items()):
                print('{}:{:0.4f}'.format(name, value), end=',  ')
            print()

        if epoch % self.dot_every == 0:
            print('.', end='', flush=True)

* Και τώρα προχωράμε στην εκπαίδευση.

In [ ]:
early_stop = [
    EpochDots(),
    tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=200),
]

history = model.fit(
    X_train, 
    y_train,
    epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.1,
    verbose=0,
    callbacks=early_stop)

* Μπορούμε να δούμε την εξέλιξη αν φτιάξουμε μια βοηθητική συνάρτηση για την απεικόνιση.

In [ ]:
def plot_loss(history):
    plt.plot(history.history['binary_crossentropy'], label='training')
    plt.plot(history.history['val_binary_crossentropy'], label='validation')
    plt.xlabel('Epoch')
    plt.ylabel('Binary Crossentropy')
    plt.legend()
    plt.grid(True)

In [ ]:
plot_loss(history)

* Τα αποτελέσματα είναι τραγωδία.

* Πολύ γρήγορα η απώλεια στην επικύρωση εκτοξεύεται.

* Αυτό είναι σαφής ένδειξη υπερπροσαρμογής.

* Ένας τρόπος να καταπολεμήσουμε την υπεπροσαρμογή στα νευρωνικά δίκτυα είναι μέσω του *dropout*, το οποίο ίσως στα ελληνικά θα μπορούσαμε να αποδώσουμε ως «θερισμός».

* Η [ιδέα πίσω από το dropout](https://en.wikipedia.org/wiki/Convolutional_neural_network#Dropout) είναι ότι σε κάθε βήμα της εκπαίδευσης, οι κόμβοι «απορρίπτονται» με πιθανότητα $p$ ή διατηρούνται με πιθανότητα $1 - p$.

* Στην πράξη δηλαδή, θερίζουμε από το νευρωνικό δίκτυο τους κόμβους με πιθανότητα $p$.

* Στο επόμενο βήμα της εκπαίδευσης, οι κόμβοι που θα απορριφθούν πάλι επιλέγονται τυχαία, άρα κάποιοι θα ξαναεισαχθούν και κάποιοι άλλοι θα βγουν.

In [ ]:
model = keras.Sequential([
    normalizer,
    layers.Dense(300, input_shape=(X_train.shape[1],), activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(300, activation='elu'),
    layers.Dropout(0.5),    
    layers.Dense(300, activation='elu'),
    layers.Dropout(0.5),    
    layers.Dense(300, activation='elu'),
    layers.Dropout(0.5),    
    layers.Dense(300, activation='elu'),
    layers.Dropout(0.5),    
    layers.Dense(1, activation='sigmoid')
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(lr_schedule),
              metrics=[
                  tf.keras.losses.BinaryCrossentropy(from_logits=False, name='binary_crossentropy'),
                  tf.metrics.BinaryAccuracy(threshold=0.5)
              ])

In [ ]:
history = model.fit(
    X_train, 
    y_train,
    epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.1,
    verbose=0,
    callbacks=early_stop)

* Βλέπουμε ότι η κατάσταση έχει κάπως βελτιωθεί.

In [ ]:
plot_loss(history)

* Μια άλλη ιδέα για να αποφύγουμε την υπερπροσαρμογή είναι η *ομαλοποίηση* (regularization).

* Στην προσέγγιση αυτή, ελαττώνουμε τα βάρη προς το μηδέν. 

* Ο λόγος είναι ότι όσο λιγότερα βάρη στο τελικό μοντέλο, τόσο λιγότερες παραμέτρους έχουμε στο τέλος, άρα τόσο πιο λιτό είναι το τελικό μοντέλο.

* Στην ομαλοποίηση L1, προσθέτουμε στην απώλεια μια ποσότητα ανάλογη με την απόλυτη τιμή του κάθε βάρους. Το αποτέλεσμα είναι να μηδενίζονται κάποια βάρη.

* Στην ομαλοποίηση L2, προσθέτουμε στην απώλεια μια ποσότητα ανάλογη με το τετράγωνο του κάθε βάρους. Το αποτέλεσμα είναι να ελαχιστοποιούνται (αλλά όχι να μηδενίζονται απολύτως) κάποια βάρη.

* Ειδικότερα, αν χρησιμοποιήσουμε `regularizers.l2(0.001)`, όπως παρακάτω, κάθε βάρος θα προσθέσει $0{,}001 \times w^2$ στην απώλεια του δικτύου.

* Αυτός είναι και ο λόγος που παρακολουθούμε την μετρική `binary_crossentropy` και όχι απλώς την απώλεια, γιατί δεν θέλουμε να λαμβάνουμε υπόψη την ομαλοποίηση.

* Τώρα στην έξοδο, τα `binary_crossentropy`, `val_binary_cross_entropy` θα διαφέρουν από τα `loss` και `val_loss`.

In [ ]:
model = keras.Sequential([
    normalizer,
    layers.Dense(300, input_shape=(X_train.shape[1],),
                 kernel_regularizer=regularizers.l2(0.001),
                 activation='elu'),
    layers.Dense(300,
                 kernel_regularizer=regularizers.l2(0.001),
                 activation='elu'),
    layers.Dense(300,
                 kernel_regularizer=regularizers.l2(0.001),
                 activation='elu'),
    layers.Dense(300,
                 kernel_regularizer=regularizers.l2(0.001),
                 activation='elu'),
    layers.Dense(300,
                 kernel_regularizer=regularizers.l2(0.001),
                 activation='elu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(lr_schedule),
              metrics=[
                  tf.keras.losses.BinaryCrossentropy(from_logits=False, name='binary_crossentropy'),
                  tf.metrics.BinaryAccuracy(threshold=0.5)
              ])

In [ ]:
history = model.fit(
    X_train, 
    y_train,
    epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.1,
    verbose=0,
    callbacks=early_stop)

* Βλέπουμε ότι και η ομαλοποίηση μπορεί να βοηθήσει.

In [ ]:
plot_loss(history)

* Οπότε, το επόμενο λογικό βήμα είναι να συνδυάσουμε και τις δύο μεθόδους για την αποφυγή της υπερπροσαρμογής.

In [ ]:
model = keras.Sequential([
    normalizer,
    layers.Dense(300, input_shape=(X_train.shape[1],),
                 kernel_regularizer=regularizers.l2(0.001),
                 activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(300,
                 kernel_regularizer=regularizers.l2(0.001),
                 activation='elu'),
    layers.Dropout(0.5),    
    layers.Dense(300,
                 kernel_regularizer=regularizers.l2(0.001),
                 activation='elu'),
    layers.Dropout(0.5),    
    layers.Dense(300,
                 kernel_regularizer=regularizers.l2(0.001),
                 activation='elu'),
    layers.Dropout(0.5),    
    layers.Dense(300,
                 kernel_regularizer=regularizers.l2(0.001),
                 activation='elu'),
    layers.Dropout(0.5),    
    layers.Dense(1, activation='sigmoid')
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(lr_schedule),
              metrics=[
                  tf.keras.losses.BinaryCrossentropy(from_logits=False, name='binary_crossentropy'),
                  tf.metrics.BinaryAccuracy(threshold=0.5)
              ])

In [ ]:
history = model.fit(
    X_train, 
    y_train,
    epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.1,
    verbose=0,
    callbacks=early_stop)

* Πετύχαμε ακόμα καλύτερη συμπεριφορά.

In [ ]:
plot_loss(history)

* Μπορούμε επιπλέον να αξιολογήσουμε το μοντέλο μας με βάση τα δεδομένα ελέγχου.

In [ ]:
metrics = model.evaluate(X_test, y_test, verbose=0)
for metric_name, metric in zip(model.metrics_names, metrics):
    print(metric_name, metric)

* Η εφαρμογή σωστής στρατηγικής για την αποφυγή υπερπροσαρμογής είναι βασικό συστατικό επιτυχίας σε ένα μοντέλο νευρωνικού δικτύου.

* Τα εργαλεία μας δίνουν τη δυνατότητα να στοιβάζουμε στρώματα επί στρωμάτων.

* Αυτό δεν σημαίνει ότι έτσι θα λύσουμε το πρόβλημα.

* Πρέπει να προσέχουμε πάντα να χρησιμοποιούμε σωστά τη δύναμη που μας δίνουν τα εργαλεία που έχουμε στα χέρια μας.